In [ ]:
import requests
import json
import os
import pandas as pd
# For parsing URLs:
from urllib.parse import quote_plus
import nltk
from langdetect import detect


In [ ]:
def search_cc_index(url, index_name):
    """
    Search the Common Crawl Index for a given URL.
 
    This function queries the Common Crawl Index <a href="https://www.jcchouinard.com/api/">API</a> to find records related to the specified URL. 
    It uses the index specified by `index_name` to retrieve the data and returns a list of JSON objects, 
    each representing a record from the index.
 
    Arguments:
        url (str): The URL to search for in the Common Crawl Index.
        index_name (str): The name of the Common Crawl Index to search (e.g., "CC-MAIN-2024-10").
 
    Returns:
        list: A list of JSON objects representing records found in the Common Crawl Index. 
              Returns None if the request fails or no records are found.
 
    Example:
        >>> search_cc_index("example.com", "CC-MAIN-2024-10")
        [{...}, {...}, ...]
    """
    encoded_url = quote_plus(url)
    index_url = f'http://index.commoncrawl.org/{index_name}-index?url={encoded_url}&output=json'
    response = requests.get(index_url)
 
    if response.status_code == 200:
        records = response.text.strip().split('\n')
        return [json.loads(record) for record in records]
    else:
        return None
 

In [ ]:
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
import sys
import nltk
from langdetect import detect
import re
import pandas as pd

# Regular expression to detect non-Latin characters
non_latin_pattern = re.compile(r'[^\x00-\x7F]+')

allowed_domains = {'com', 'gov', 'edu', 'co', 'uk', 'net', 'mil', 'ai', 'ca'}

def is_english(text):
    try:
        return detect(text) == 'en' and not non_latin_pattern.search(text)
    except:
        return False

def is_latin_not_english(text):
    try:
        return detect(text) != 'en' and not non_latin_pattern.search(text)
    except:
        return False


def get_last_domain_part(url:str):
    return url.split("/")[2].split(".")[-1]

def is_error_response(input:str):
    block_words = {"404"}
    input = input.lower()
    words = input.split()
    for word in words:
        if word in block_words:
            return True
    if input.find("no response") >=0:
        return True
    if input.find("not found") >=0:
        return True
    return False
    

def extract_english_files(warc_file):
    """
    Returns a list of dictionaries with url description and title keys from 
    english web pges in a WARC file
    """
    count = 0
    results = []
    with open(warc_file, 'rb') as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type == 'response' and 'text/html' in record.http_headers.get('Content-Type', ''):
                payload = record.content_stream().read()
                soup = BeautifulSoup(payload, 'html.parser')
                
                html_tag = soup.find('html')
                if html_tag and html_tag.get('lang', '').startswith('en'):
                    url = record.rec_headers.get('WARC-Target-URI')
                    if not get_last_domain_part(url) in allowed_domains:
                        continue
                    title_tag = soup.find('title')
                    title = title_tag.text.strip() if title_tag else None
                    if title is None or is_error_response(title):
                        continue
                    og_desc_tag = soup.find('meta', attrs={'property': 'og:description'})
                    description = og_desc_tag.get('content', '').strip() if og_desc_tag else None
                    if description is None:
                        meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
                        description = meta_desc_tag.get('content', '').strip() if meta_desc_tag else 'No Description'
                    
                    if not is_english(title):
                        continue
                    if count%20 == 0:
                        print(count)
                    count += 1
                    results.append({"url": url, "description": description, "title": title})
    return results

def extract_non_english_latin(warc_file):
    """
    Extracts files that have latin charsets in the title but a language detector determines as non-english
    This is good for exracting error pages, pages in non-english languages
    """
    count = 0
    results = []
    with open(warc_file, 'rb') as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type == 'response' and 'text/html' in record.http_headers.get('Content-Type', ''):
                payload = record.content_stream().read()
                soup = BeautifulSoup(payload, 'html.parser')
                
                html_tag = soup.find('html')
                if html_tag and not html_tag.get('lang', '').startswith('en'):
                    url = record.rec_headers.get('WARC-Target-URI')
                    if not get_last_domain_part(url) in allowed_domains:
                        continue
                    title_tag = soup.find('title')
                    title = title_tag.text.strip() if title_tag else None
                    if title is None or is_error_response(title):
                        continue
                    og_desc_tag = soup.find('meta', attrs={'property': 'og:description'})
                    description = og_desc_tag.get('content', '').strip() if og_desc_tag else None
                    if description is None:
                        meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
                        description = meta_desc_tag.get('content', '').strip() if meta_desc_tag else 'No Description'
                    
                    if not is_latin_not_english(title):
                        continue
                    if count%20 == 0:
                        print(count)
                    count += 1
                    if count > 800:
                        break
                    results.append({"url": url, "description": description, "title": title})
    return results    



In [ ]:
r = extract_english_files('/Users/Rrando/crawl/out/CC-MAIN-20250218081924-20250218111924-00893.warc.gz')
cc_corpus = pd.DataFrame(r)
cc_corpus.to_csv("../data/external/common_crawl.csv")

In [ ]:
r = extract_non_english_latin('/Users/Rrando/crawl/out/CC-MAIN-20250218081924-20250218111924-00893.warc.gz')
cc_corpus = pd.DataFrame(r)
cc_corpus.to_csv("../data/external/common_crawl_non_english.csv")